## ○貪欲法＆動的計画法による複数ナップザック問題の近似解推定

In [1]:
import time
import cvxpy
import random
import numpy as np
import pandas as pd

In [2]:
# サンプルデータの読み込み
df_sample = pd.read_csv('sample_10*5000.csv', sep='\t')

In [3]:
df_sample.head(3)

,id,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,...,rating_0,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9
0,1,380,385,389,375,387,385,382,378,385,...,4.1,3.3,4.0,3.5,3.8,4.5,4.1,3.8,3.8,3.8
1,2,943,947,961,960,960,947,950,940,944,...,10.1,8.9,9.3,10.3,9.4,9.5,9.9,9.4,8.4,9.1
2,3,980,980,987,982,974,982,981,982,975,...,10.4,8.7,10.3,9.8,9.3,9.0,9.8,9.8,9.8,9.8


In [4]:
# clientの数。
client_num = int((len(df_sample.columns)-1)/2)
print('クライアントの数:', client_num)

クライアントの数: 10


In [5]:
# 初めにクライアントが持っていた枠をランダムに決める（別のデータに対応できるかを見るため。）
sample = df_sample.sample(frac=1).reset_index(drop=True)
sample.head(3)

,id,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,...,rating_0,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9
0,3030,830,827,829,837,837,829,832,832,823,...,9.8,9.8,9.0,8.0,8.0,8.0,7.1,7.1,7.7,7.9
1,1582,314,314,314,307,307,307,316,310,307,...,2.4,2.4,2.2,3.1,3.1,3.1,3.1,3.6,4.1,4.3
2,905,897,899,899,899,897,905,907,912,913,...,9.3,9.4,8.9,8.7,8.7,8.7,8.6,8.6,9.0,9.0


In [6]:
# クライアト0, 1, 2の元々持っていた枠をランダムに決める。
length = len(sample) # データフレームの長さ
bps = [0 for i in range(client_num+1)] # Break Points
bp = 0
original_prices = [0 for i in range(client_num)] # 元々持っていた枠の価格和。
original_points = [0 for i in range(client_num)] # 元々持っていた枠の視聴率和。

for i in range(client_num):
    bp += length//client_num
    bps[i+1] = bp
    df_original = sample[bps[i]:bps[i+1]]
    original_prices[i] = sum(df_original['price_' + str(i)]) # 価格の和。
    original_points[i] = sum(df_original['rating_' + str(i)]) # 視聴率の和。

In [7]:
print(original_prices) # 元々持っていた枠の価格和。
print(original_points) # 元々持っていた枠の視聴率和。

[426025, 423649, 418169, 437708, 417187, 427844, 398686, 425675, 422501, 436930]
[4277.800000000001, 4243.099999999999, 4170.400000000002, 4365.700000000003, 4174.700000000001, 4274.999999999997, 3964.599999999998, 4258.100000000005, 4222.299999999999, 4361.900000000002]


***
***

### 〜ここからナップザック問題（貪欲法のアプローチ）のアルゴリズム〜

In [8]:
# 必要な視聴率和(=元々持っていた視聴率和)
require_ratings = list(original_points) # 参照してしまうことを避けるため。
print('注意！参照されてる！！' if id(require_ratings) == id(original_points) else '参照されてない。 good!!')

参照されてない。 good!!


In [9]:
# value=「視聴率/号数（価格）」を求める。(これを比較して、貪欲に取り出す。)
for i in range(client_num):
    df_sample['value_'+str(i)] = df_sample['rating_'+str(i)]/df_sample['price_'+str(i)]

# valueができていることの確認。
df_sample.head(3)

,id,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,...,value_0,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9
0,1,380,385,389,375,387,385,382,378,385,...,0.010789,0.008571,0.010283,0.009333,0.009819,0.011688,0.010733,0.010053,0.009870,0.009948
1,2,943,947,961,960,960,947,950,940,944,...,0.010710,0.009398,0.009677,0.010729,0.009792,0.010032,0.010421,0.010000,0.008898,0.009499
2,3,980,980,987,982,974,982,981,982,975,...,0.010612,0.008878,0.010436,0.009980,0.009548,0.009165,0.009990,0.009980,0.010051,0.009990


<b><font color='Red'>※valueは、正の数じゃなくても良い（単なる大きさ比較だから。）</font>
    <br>しかし、余ったものをナップザック問題で解くので、そこでは正の数でないといけない。（０の方が大きいから。）
    <br>とはいえ、価格、視聴率のどちらかが負の数になることは考えにくいため、意識するる必要はなさげ。</b>

In [10]:
def Greedy_optimizer(df, require_ratings, num):
    check_point=0 # 目的を達成した（元々の視聴率和を超えた）クライアントの数。
    require_points = list(require_ratings) # 元のリストが変わらないようにする。
    df = df.copy() # 元のデータフレームが変わらないようにする。
    
    value_col_names = ['value_'+str(i) for i in range(num)] # valueのカラム名（ここから目的を達成したクライアントを取り除いていく。）
    get_ids = dict() # 獲得した枠のidを格納する。
    for i in range(num):
        get_ids[i] = []
    
    while check_point < num:
        df['Max_value'] = df.loc[:, value_col_names].max(axis=1) # (残っている)クライアントの価値の中で最も高い値。
        df['Max_value_client'] = df.loc[:, value_col_names].idxmax(axis=1).apply(lambda x:x.replace('value_', '')) # 最も高い価値をつけているクライアントの名前。
        df = df.sort_values(by='Max_value', ascending=False).reset_index(drop=True) # その価値でソートする。
        
        for i in range(len(df)):
            name = int(df.iat[0, -1]) # 最も高い値をつけているクライアントの名前。
            require_points[name] -= df.at[0, 'rating_'+str(name)] # 必要な視聴率和からその枠の視聴率を引く。
            get_ids[name] += [df.iat[0, 0]] # 獲得した枠のidを記録する。
            df = df[1:].reset_index(drop=True) # dfを下っていく。
            if min(require_points) < 0: # どこかのクライアントが、目的を達成したら、一回終わり。
                break

        min_client = require_points.index(min(require_points)) # 目的を達成したクライアント名。（番号）
        require_points[min_client] = 0 # 目的を達成したら、0にする。（今後のため。）
        drop_col_name = ['price_'+str(min_client), 'rating_'+str(min_client), 'value_'+str(min_client)]
        value_col_names.remove(drop_col_name[-1]) # 目的を達成したクライアントは、除く。

        check_point += 1 # チェックポイント＝目的を達成したクライアントの数。
            
    return list(get_ids.values())

In [11]:
# 2重のリストをフラットにする関数(重複は残る！)
def Flatten_dual(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

In [12]:
from ortoolpy import knapsack

start = time.time() # プログラム開始時間

obtained_idses = Greedy_optimizer(df_sample, require_ratings, client_num) # 獲得した枠のid
obtained_prices = [0 for i in range(client_num)] # 獲得した枠の価格和。
obtained_points = [0 for i in range(client_num)] # 獲得した枠の視聴率和。
rate = 1.05 # どれだけ価格の超過分を許容するか。

for i in range(client_num):
    df_sample['judge'] = df_sample['id'].apply(lambda x:x in obtained_idses[i]) # 各クライアントが獲得した枠を判断するカラム。
    result = df_sample[df_sample['judge']] 
    obtained_prices[i] = sum(result['price_' + str(i)]) # 獲得した枠の合計額。
    obtained_points[i] = sum(result['rating_' + str(i)]) # 獲得した枠の合計視聴率。
    
all_get_ids = Flatten_dual(obtained_idses) # クライアントが獲得したアカウント全てを足し合わせたもの。
df_sample['judge'] = df_sample['id'].apply(lambda x:x not in all_get_ids) # 残った枠を判別するカラム。
remain_df = df_sample[df_sample['judge']].reset_index(drop=True) # 残ったデータフレーム。
capacities = list(np.array(original_prices) * rate - np.array(obtained_prices)) # 元々持っていた枠の合計価格(の1.05倍)から獲得した枠の合計価格を引く。

print('*'*20, '貪欲法によるナップザック問題の解法の結果', '*'*20)
print('交換前の合計価格　:', original_prices)
print('獲得枠の合計価格　:', obtained_prices)
print('交換前の合計視聴率:', np.round(np.array(original_points), 2))
print('獲得枠の合計視聴率:', np.round(np.array(obtained_points), 2))
print('残りキャパシティー:', np.round(np.array(capacities), 2))
print('残り枠数:', len(remain_df))
display(remain_df.head(3))
end = time.time() # プログラム終了時間
print('*'*24, 'プログラム処理経過時間', round(end-start,5), '[sec]', '*'*24)
    
count = 1
while len(remain_df)>0:
    capacities = list(np.array(original_prices) * rate - np.array(obtained_prices)) # 元々持っていた枠の合計価格(の1.05倍)から獲得した枠の合計価格を引く。
    add_idses = [[] for i in range(client_num)] # 各ナップザック問題の結果、獲得した枠のidを格納する。
    add_ids = [0 for i in range(client_num)]
    
    for i in range(client_num):
        size = list(np.array(remain_df['price_' + str(i)]))
        weight = list(np.array(remain_df['rating_' + str(i)]))
        capacity = capacities[i]
        add_ids[i] = knapsack(size, weight, capacity)[1]
        
    ids_dict = dict()
    for key in range(len(remain_df)): # それぞれのindex
        values = [] # そのidの枠を指定したクライアントを格納するリスト。
        for j in range(client_num): # それぞれのクライアントごとに、
            if key in add_ids[j]: # そのid（のindex）があれば、
                values.append(j) # valuesにクライアントの名前が格納される。
        if values: # そのidを指定したクライアントがいれば、
            value = random.choice(values) # その中からクライアントをランダムに選ぶ。
            ids_dict[key] = value # idと合わせて格納される。
            
    if len(ids_dict)==0:
        rate += 0.05*(count)
        count += 1
                
    for i in range(client_num):
        add_idses[i] = [ids for ids, client in ids_dict.items() if client == i] # クライアントごとに、獲得したid
        # obtained_idses[i] += add_idses[i]
        obtained_idses[i] += remain_df.query('index in ' + str(add_idses[i]))['id'].values.tolist()
        obtained_prices[i] += sum(remain_df.query('index in ' + str(add_idses[i]))['price_' + str(i)])
        obtained_points[i] += sum(remain_df.query('index in ' + str(add_idses[i]))['rating_' + str(i)])

    extraction_ids = Flatten_dual(add_idses) # 今回獲得されたidの集合を、取り除く。
    remain_df = remain_df.query('index not in ' + str(extraction_ids)).reset_index(drop=True)
    
print('*'*20, '動的計画法によるナップザック問題の解法の結果', '*'*20)
print('交換前の合計価格　:', original_prices)
print('獲得枠の合計価格　:', obtained_prices)
print('交換前の合計視聴率:', np.round(np.array(original_points), 2))
print('獲得枠の合計視聴率:', np.round(np.array(obtained_points), 2))
print('残り枠数:', len(remain_df))
display(remain_df.head(3))

end = time.time() # プログラム終了時間
print('*'*24, 'プログラム処理経過時間', round(end-start,5), '[sec]', '*'*24)

******************** 貪欲法によるナップザック問題の解法の結果 ********************
交換前の合計価格　: [426025, 423649, 418169, 437708, 417187, 427844, 398686, 425675, 422501, 436930]
獲得枠の合計価格　: [352617, 380231, 384786, 408333, 391194, 401154, 366762, 391851, 381226, 372271]
交換前の合計視聴率: [4277.8 4243.1 4170.4 4365.7 4174.7 4275.  3964.6 4258.1 4222.3 4361.9]
獲得枠の合計視聴率: [4279.8 4251.3 4171.  4366.2 4180.1 4279.8 3967.8 4260.  4237.4 4362. ]
残りキャパシティー: [94709.25 64600.45 54291.45 51260.4  46852.35 48082.2  51858.3  55107.75
 62400.05 86505.5 ]
残り枠数: 327


,id,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,...,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,judge
0,18,413,401,429,414,414,423,417,414,410,...,0.009975,0.010256,0.009903,0.008454,0.009693,0.010072,0.009662,0.010000,0.009951,True
1,29,1237,1254,1255,1249,1249,1249,1241,1255,1251,...,0.009171,0.009960,0.010088,0.009287,0.009928,0.009508,0.009880,0.009912,0.009287,True
2,37,1533,1533,1533,1533,1533,1533,1533,1542,1541,...,0.009980,0.009980,0.009785,0.009785,0.009785,0.010241,0.010311,0.010448,0.010110,True


************************ プログラム処理経過時間 7.84524 [sec] ************************
******************** 動的計画法によるナップザック問題の解法の結果 ********************
交換前の合計価格　: [426025, 423649, 418169, 437708, 417187, 427844, 398686, 425675, 422501, 436930]
獲得枠の合計価格　: [419961, 417736, 423299, 439000, 416223, 429378, 401429, 425079, 424542, 429201]
交換前の合計視聴率: [4277.8 4243.1 4170.4 4365.7 4174.7 4275.  3964.6 4258.1 4222.3 4361.9]
獲得枠の合計視聴率: [4984.9 4640.  4563.2 4675.4 4433.  4563.7 4322.2 4600.4 4688.2 4968.8]
残り枠数: 0


,id,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,...,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,judge


************************ プログラム処理経過時間 10.5446 [sec] ************************


<b>ここからやらなければならないこと。
    1. 価格を超えてしまった時の対処法。（処理時間と要相談）
    2. プログラムの見直しと改善。
</b>